In [24]:
import pandas as pd
import sqlite3
import gc
import itertools

from functools import lru_cache

# Calculate reference MidPrices for random moments

In [5]:
# TRADES_DIR = '../TRADES/USD_TOM_trades_Micex_02_10.feather'
LOB_DIR = '../Data_lob/'

# RANDOM_MOMENTS_ALIGNED_OUTPUT = '../0_CommonFiles/random_moments_aligned_to_1000_2000.pkl'
TOXIC_BUY_MOMENTS_PATH = '../0_CommonFiles/toxic/toxic_buy_moments_1m_02_06.feather'
TOXIC_SELL_MOMENTS_PATH = '../0_CommonFiles/toxic/toxic_sell_moments_1m_02_06.feather'
MIDPRICE_DB_PATH = '../0_CommonFiles/mid.db3'

In [6]:
class LOBCache:
    def __init__(self):
        pass
    
    def get_lob(self, datetime):
        filename = datetime.strftime('LOB_%m%d.feather')
        return self.__get_lob_by_filename(filename)
        pass
    
    @lru_cache(maxsize=1)
    def __get_lob_by_filename(self, filename):
        print(filename)
        lob = pd.read_feather(LOB_DIR + filename).dropna()
        lob['BID_SIZE10'] = 99_999_999
        lob['ASK_SIZE10'] = 99_999_999
        return lob

In [7]:
cache = LOBCache()

# Read and prepare 'toxic' moments

In [10]:
toxic_buy_moments = pd.read_feather(TOXIC_BUY_MOMENTS_PATH)

In [11]:
toxic_buy_moments_2 = pd.DataFrame(index=toxic_buy_moments['Time']).between_time('10:00', '23:50')

In [12]:
toxic_buy_moments_2

""
Time
2021-02-01 10:01:36.771
2021-02-01 10:05:02.883
2021-02-01 10:05:11.061
2021-02-01 10:05:23.526
2021-02-01 10:07:27.929
...
2021-06-30 19:17:20.554
2021-06-30 19:30:16.189
2021-06-30 19:31:46.414


In [13]:
toxic_buy_moments_2_index = toxic_buy_moments_2.index

In [14]:
toxic_sell_moments = pd.read_feather(TOXIC_SELL_MOMENTS_PATH)

In [15]:
toxic_sell_moments_2 = pd.DataFrame(index=toxic_sell_moments['Time']).between_time('10:00', '23:50')

In [12]:
toxic_sell_moments_2

""
Time
2021-02-01 10:00:09.015
2021-02-01 10:04:36.332
2021-02-01 10:04:57.946
2021-02-01 10:05:19.015
2021-02-01 10:08:27.788
...
2021-06-30 19:03:41.640
2021-06-30 19:05:10.360
2021-06-30 19:05:24.713


In [16]:
toxic_sell_moments_2_index = toxic_sell_moments_2.index

# Calculation

In [22]:
#CREATE TABLE MIDPRICE(MOMENT TEXT, PRICE REAL, PRIMARY KEY(MOMENT));
mid_sql = sqlite3.connect(MIDPRICE_DB_PATH)
mid_sql.execute('CREATE TABLE IF NOT EXISTS MIDPRICE(MOMENT TEXT, PRICE REAL, PRIMARY KEY(MOMENT));')
cur=mid_sql.cursor()

In [25]:
for moment in itertools.chain(toxic_buy_moments_2_index, toxic_sell_moments_2_index):
    record = (str(moment),)
    query='select exists(select 1 from MIDPRICE where MOMENT=? collate nocase) limit 1'
    # 'query' RETURNS 1 IF USERNAME EXISTS OR 0 IF NOT, AS INTEGER(MAYBE). 'collate nocase'= CASE INSENSITIVE, IT'S OPTIONAL
    check=cur.execute(query, record) 
    if check.fetchone()[0]==1:
        print(f'Moment already available ' + str(record))
        continue

    query='INSERT OR REPLACE INTO MIDPRICE(MOMENT, PRICE) VALUES(?, ?);'
    
    lob = cache.get_lob(moment)
    index = lob.Time.searchsorted(moment, side = 'right')
    if len(lob) > 0:
        row = lob.iloc[index]
        mid = (row['BID_PRICE1'] + row['ASK_PRICE1']) / 2
        mid = round(mid, 5)
        record = (str(moment), mid)
    else:
        record = (str(moment), -1.0)
    
    print(record)
    cur.execute(query, record)
    gc.collect()

Moment already available ('2021-02-01 10:01:36.771000',)
('2021-02-01 10:05:02.883000', 75.6)
('2021-02-01 10:05:11.061000', 75.57625)
('2021-02-01 10:05:23.526000', 75.58625)
('2021-02-01 10:07:27.929000', 75.65)
('2021-02-01 10:11:51.521000', 75.5875)
('2021-02-01 10:12:14.647000', 75.61375)
('2021-02-01 10:13:16.218000', 75.6425)
('2021-02-01 10:13:16.224000', 75.6425)
('2021-02-01 10:13:18.165000', 75.645)
('2021-02-01 10:14:31.507000', 75.66)
('2021-02-01 10:15:58.571000', 75.645)
('2021-02-01 10:17:42.974000', 75.59625)
('2021-02-01 10:17:51.119000', 75.5975)
('2021-02-01 10:19:06.504000', 75.54875)
('2021-02-01 10:22:57.488000', 75.54625)
('2021-02-01 10:24:58.090000', 75.545)
('2021-02-01 10:27:49.696000', 75.51625)
('2021-02-01 10:30:01.974000', 75.46)
('2021-02-01 10:30:02.098000', 75.46375)
('2021-02-01 10:31:15.367000', 75.44375)
('2021-02-01 10:32:18.384000', 75.40875)
('2021-02-01 10:37:27.007000', 75.4425)
('2021-02-01 10:42:43.364000', 75.46)
('2021-02-01 10:52:34.41700

('2021-02-01 21:12:58.262000', 76.19)
('2021-02-01 21:12:58.822000', 76.19)
('2021-02-01 21:13:08.022000', 76.1875)
LOB_0202.feather
('2021-02-02 10:00:01.504000', 75.93)
('2021-02-02 10:00:54.981000', 75.925)
('2021-02-02 10:01:51.175000', 75.88875)
('2021-02-02 10:02:06.109000', 75.90125)
('2021-02-02 10:02:06.187000', 75.90375)
('2021-02-02 10:02:26.397000', 75.9125)
('2021-02-02 10:11:34.713000', 75.865)
('2021-02-02 10:13:07.015000', 75.9)
('2021-02-02 10:13:41.934000', 75.8975)
('2021-02-02 10:13:42.227000', 75.89875)
('2021-02-02 10:13:59.885000', 75.895)
('2021-02-02 10:16:05.958000', 75.96125)
('2021-02-02 10:18:57.100000', 76.01625)
('2021-02-02 10:21:33.942000', 76.045)
('2021-02-02 10:23:43.822000', 76.06)
('2021-02-02 10:53:54.621000', 75.90875)
('2021-02-02 10:57:27.488000', 75.9525)
('2021-02-02 11:05:24.134000', 75.91125)
('2021-02-02 11:07:06.280000', 75.90625)
('2021-02-02 11:07:25.244000', 75.91625)
('2021-02-02 11:17:46.681000', 75.7625)
('2021-02-02 11:19:16.341000

('2021-02-02 18:57:15.177000', 75.6275)
('2021-02-02 19:00:01.018000', 75.645)
('2021-02-02 19:01:21.125000', 75.675)
('2021-02-02 19:04:47.126000', 75.66)
('2021-02-02 19:05:46.444000', 75.64875)
('2021-02-02 19:09:20.076000', 75.59375)
('2021-02-02 19:10:14.756000', 75.52875)
('2021-02-02 19:10:51.505000', 75.56375)
('2021-02-02 19:11:54.351000', 75.56125)
('2021-02-02 19:12:23.707000', 75.5525)
('2021-02-02 19:17:18.576000', 75.60625)
('2021-02-02 19:17:33.065000', 75.57125)
('2021-02-02 19:23:30.975000', 75.64625)
('2021-02-02 19:26:02.810000', 75.62875)
('2021-02-02 19:47:18.446000', 75.645)
('2021-02-02 19:50:06.477000', 75.655)
('2021-02-02 19:50:06.556000', 75.655)
('2021-02-02 19:58:13.106000', 75.7175)
('2021-02-02 20:15:51.030000', 75.76125)
('2021-02-02 20:21:55.346000', 75.85625)
('2021-02-02 20:22:10.717000', 75.905)
('2021-02-02 20:22:10.720000', 75.905)
('2021-02-02 20:29:28.313000', 75.8525)
('2021-02-02 20:29:28.432000', 75.8525)
('2021-02-02 20:29:33.265000', 75.8462

('2021-02-03 18:55:53.929000', 75.95375)
('2021-02-03 18:56:32.785000', 75.9775)
('2021-02-03 18:57:29.934000', 76.015)
('2021-02-03 18:57:42.161000', 76.02375)
('2021-02-03 18:57:42.262000', 76.02375)
('2021-02-03 19:00:29.421000', 76.02)
('2021-02-03 19:03:45.078000', 75.985)
('2021-02-03 19:04:15.639000', 75.96375)
('2021-02-03 19:04:39.831000', 75.9475)
('2021-02-03 19:04:43.592000', 75.945)
('2021-02-03 19:04:48.789000', 75.93375)
('2021-02-03 19:13:04.546000', 76.02)
('2021-02-03 19:17:14.664000', 76.07875)
('2021-02-03 19:17:14.717000', 76.08)
('2021-02-03 19:18:01.204000', 76.06375)
('2021-02-03 19:18:07.371000', 76.06125)
('2021-02-03 19:26:24.297000', 76.04625)
('2021-02-03 19:27:26.309000', 76.03125)
('2021-02-03 19:34:50.245000', 75.96875)
('2021-02-03 19:38:17.194000', 75.94125)
('2021-02-03 19:39:37.251000', 75.9175)
('2021-02-03 19:43:19.811000', 75.8775)
('2021-02-03 19:43:57.447000', 75.89375)
('2021-02-03 22:20:50.130000', 75.8475)
('2021-02-03 22:20:50.153000', 75.84

('2021-02-04 20:21:52.112000', 75.7025)
('2021-02-04 20:24:52.968000', 75.72375)
('2021-02-04 20:24:52.998000', 75.72375)
('2021-02-04 20:32:20.488000', 75.79125)
('2021-02-04 20:35:24.551000', 75.78625)
('2021-02-04 20:35:38.550000', 75.7825)
('2021-02-04 20:42:22.308000', 75.79125)
('2021-02-04 22:03:30.089000', 75.67)
('2021-02-04 22:45:44.290000', 75.59625)
('2021-02-04 22:45:44.410000', 75.59625)
LOB_0205.feather
('2021-02-05 10:01:54.290000', 75.28625)
('2021-02-05 10:03:16.561000', 75.25375)
('2021-02-05 10:03:16.623000', 75.2575)
('2021-02-05 10:04:16.078000', 75.23)
('2021-02-05 10:04:26.990000', 75.20625)
('2021-02-05 10:05:01.687000', 75.20875)
('2021-02-05 10:05:47.713000', 75.13875)
('2021-02-05 10:06:16.692000', 75.16)
('2021-02-05 10:08:44.907000', 75.2175)
('2021-02-05 10:12:14.094000', 75.21125)
('2021-02-05 10:13:41.382000', 75.2125)
('2021-02-05 10:16:00.938000', 75.1425)
('2021-02-05 10:16:03.056000', 75.13625)
('2021-02-05 10:16:54.117000', 75.15375)
('2021-02-05 1

...

('2021-06-18 10:02:23.558000', 72.37875)
('2021-06-18 10:11:39.568000', 72.2275)
('2021-06-18 10:25:42.186000', 72.19625)
('2021-06-18 10:28:56.923000', 72.195)
('2021-06-18 10:31:09.619000', 72.16625)
('2021-06-18 10:34:07.572000', 72.16375)
('2021-06-18 10:36:09.258000', 72.16125)
('2021-06-18 10:51:03.801000', 72.2325)
('2021-06-18 10:53:37.755000', 72.2125)
('2021-06-18 10:57:06.215000', 72.195)
('2021-06-18 10:58:40.481000', 72.225)
('2021-06-18 11:09:57.307000', 72.24875)
('2021-06-18 11:16:24.645000', 72.19375)
('2021-06-18 11:18:11.107000', 72.15375)
('2021-06-18 11:18:52.200000', 72.135)
('2021-06-18 11:27:10.183000', 72.15)
('2021-06-18 11:29:51.463000', 72.1325)
('2021-06-18 11:37:20.711000', 72.23375)
('2021-06-18 11:40:30.119000', 72.23)
('2021-06-18 11:45:10.239000', 72.21375)
('2021-06-18 12:13:18.913000', 72.19625)
('2021-06-18 12:17:25.196000', 72.18375)
('2021-06-18 12:25:21.922000', 72.1725)
('2021-06-18 12:26:40.730000', 72.13875)
('2021-06-18 12:26:46.821000', 72.1

Moment already available ('2021-06-21 16:28:13.681000',)
('2021-06-21 16:35:33.041000', 73.17375)
Moment already available ('2021-06-21 16:35:33.041000',)
('2021-06-21 16:39:40.639000', 73.13375)
Moment already available ('2021-06-21 16:39:40.639000',)
('2021-06-21 16:39:40.721000', 73.13375)
Moment already available ('2021-06-21 16:39:40.721000',)
('2021-06-21 16:44:21.061000', 73.195)
Moment already available ('2021-06-21 16:44:21.061000',)
('2021-06-21 17:04:45.106000', 73.15125)
Moment already available ('2021-06-21 17:04:45.106000',)
('2021-06-21 17:12:01.260000', 73.1475)
Moment already available ('2021-06-21 17:12:01.260000',)
('2021-06-21 17:13:20.822000', 73.15875)
Moment already available ('2021-06-21 17:13:20.822000',)
('2021-06-21 17:33:37.190000', 73.10625)
Moment already available ('2021-06-21 17:33:37.190000',)
('2021-06-21 17:34:25.347000', 73.06875)
Moment already available ('2021-06-21 17:34:25.347000',)
('2021-06-21 17:36:32.675000', 73.00875)
Moment already availabl

('2021-06-23 13:55:50.910000', 72.7)
('2021-06-23 13:57:53.210000', 72.73)
('2021-06-23 14:00:37.476000', 72.79)
('2021-06-23 14:00:39.943000', 72.795)
('2021-06-23 14:01:13.609000', 72.79625)
('2021-06-23 14:01:32.532000', 72.785)
('2021-06-23 14:11:40.345000', 72.715)
('2021-06-23 14:18:08.224000', 72.72625)
('2021-06-23 14:20:59.531000', 72.69875)
('2021-06-23 14:33:12.889000', 72.6725)
('2021-06-23 14:36:01.335000', 72.6525)
('2021-06-23 14:39:17.681000', 72.645)
('2021-06-23 14:47:55.740000', 72.62375)
('2021-06-23 14:55:38.194000', 72.65875)
('2021-06-23 15:04:28.126000', 72.6275)
('2021-06-23 15:04:39.045000', 72.63)
('2021-06-23 15:11:18.911000', 72.58)
('2021-06-23 15:11:25.052000', 72.5825)
('2021-06-23 15:20:19.880000', 72.50875)
('2021-06-23 15:22:45.703000', 72.54125)
('2021-06-23 15:27:29.526000', 72.55375)
('2021-06-23 15:33:30.883000', 72.505)
('2021-06-23 15:40:08.972000', 72.51625)
('2021-06-23 15:45:58.719000', 72.50125)
('2021-06-23 15:54:20.402000', 72.5175)
('2021

('2021-06-25 16:03:12.760000', 72.105)
('2021-06-25 16:19:00.144000', 72.0575)
('2021-06-25 16:36:42.381000', 72.12125)
('2021-06-25 17:21:10.148000', 72.15625)
('2021-06-25 18:02:16.884000', 72.205)
('2021-06-25 18:43:48.593000', 72.17625)
('2021-06-25 18:51:46.596000', 72.20125)
('2021-06-25 18:51:46.825000', 72.20125)
('2021-06-25 18:51:46.923000', 72.19875)
('2021-06-25 20:07:43.839000', 72.21375)
('2021-06-25 20:58:57.044000', 72.26375)
('2021-06-25 22:38:22.517000', 72.25)
('2021-06-25 22:38:22.545000', 72.25)
('2021-06-25 22:46:41.443000', 72.22625)
LOB_0628.feather
('2021-06-28 10:04:24.610000', 72.16625)
('2021-06-28 10:05:36.935000', 72.145)
('2021-06-28 10:08:34.870000', 72.1675)
('2021-06-28 10:14:11.331000', 72.17875)
('2021-06-28 10:17:26.341000', 72.18375)
('2021-06-28 10:36:28.747000', 72.13625)
('2021-06-28 10:55:57.702000', 72.2)
('2021-06-28 11:08:27.688000', 72.1725)
('2021-06-28 11:12:44.078000', 72.21875)
('2021-06-28 11:17:58.090000', 72.215)
('2021-06-28 11:20:0

('2021-06-30 15:17:00.774000', 72.99375)
('2021-06-30 15:19:21.173000', 72.95125)
('2021-06-30 15:19:36.987000', 72.94875)
('2021-06-30 15:27:20.951000', 72.98375)
('2021-06-30 15:28:59.852000', 73.03)
('2021-06-30 15:32:16.059000', 72.98375)
('2021-06-30 15:32:23.507000', 72.98375)
('2021-06-30 15:32:40.819000', 72.98625)
('2021-06-30 15:33:39.685000', 72.9875)
('2021-06-30 15:49:34.585000', 72.94625)
('2021-06-30 15:55:10.477000', 72.905)
('2021-06-30 16:04:31.941000', 72.875)
('2021-06-30 16:05:05.211000', 72.86375)
('2021-06-30 16:09:44.515000', 72.91125)
('2021-06-30 16:32:03.575000', 72.9075)
('2021-06-30 16:47:08.656000', 72.96625)
('2021-06-30 16:57:34.437000', 72.8975)
('2021-06-30 16:59:30.937000', 72.90125)
('2021-06-30 17:11:03.996000', 72.91125)
('2021-06-30 17:32:55.386000', 72.97)
('2021-06-30 17:33:53.218000', 72.9575)
('2021-06-30 17:37:22.895000', 72.9075)
('2021-06-30 17:40:01.838000', 72.905)
('2021-06-30 17:40:07.211000', 72.915)
('2021-06-30 17:40:42.290000', 72.9

In [26]:
mid_sql.commit()

In [27]:
cur.close()
mid_sql.close()